In [111]:
def process_steps(steps, initialVariables,variables):
    for step in steps:
        if 'script' in step:
            script_content = step['script']
            expanded_script = expand_variables(script_content, variables, initialVariables)
            # print(f"Executing script:\n{expanded_script}\n")
            execute_script(expanded_script)
        elif 'bash' in step:
            bash_command = step['bash']
            expanded_bash_command = expand_variables(bash_command, variables, initialVariables)
            print(f"Executing bash command: {expanded_bash_command}\n")
            update_variable_from_bash(expanded_bash_command, variables)
            # execute_bash_command(expanded_bash_command)
        else:
            print("Unsupported step type")

def expand_variables(content, variables, initialVariables):
    for key in variables:
        content = re.sub(fr'\${{\s*{{\s*variables.{key}\s*}}\s*}}', str(initialVariables[key]), content)
        content = re.sub(fr"\$\(\s*{key}\s*\)", str(variables[key]), content)
    return content

def update_variable_from_bash(bash_command, variables):
    match = re.search(r"\[task\.setvariable variable=(\w+)\](\w+)", bash_command)
    if match:
        variable_name = match.group(1)
        new_value = match.group(2)
        for key in variables:
            if key == variable_name:
                variables[key] = new_value

def execute_script(script):
    stdout, stderr, returncode = run_docker_command("docker build -t bash-docker .")
    print(f"Build Output:\n{stdout}")
    print(f"Build Error:\n{stderr}")
    print(f"Build Return Code: {returncode}")
    
    container_id = run_docker_command("docker run -d --rm bash-docker")[0]
    print(container_id)

    stdout, stderr, returncode = run_docker_command(f"docker exec {container_id} /bin/bash -c '{script}'")

    print(f"Commands: {script}")
    print(f"Output:\n{stdout}")
    print(f"Error:\n{stderr}")
    print(f"Return Code: {returncode}")
    print("=" * 50)

    run_docker_command("docker stop {container_id}")
    run_docker_command("docker rm {container_id}")

def run_docker_command(command):
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout, result.stderr, result.returncode

# def execute_bash_command(bash_command):
#     # Use subprocess.run() to execute the bash command
#     print(subprocess.check_output(['bash','-c', bash_command]))

In [112]:
yaml_file_path = 'test.yml'
parsed_data = parse_yaml_script_file(yaml_file_path)

if(parsed_data):
    initialVariables = {}
    for key in parsed_data.get('variables', []):
        initialVariables[key.get('name')] = key.get('value')
    variables = initialVariables.copy()

    steps = parsed_data.get('steps', [])
    process_steps(steps, initialVariables, variables)

Build Output:

Build Error:
#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile:
#1 transferring dockerfile: 127B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:latest
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:latest
#3 DONE 1.3s

#5 [1/2] FROM docker.io/library/ubuntu:latest@sha256:2b7412e6465c3c7fc5bb21d3e6f1917c167358449fecac8176c6e496e5c1f05f
#5 DONE 0.0s

#6 [2/2] RUN apt-get update && apt-get install -y     bash
#6 CACHED

#7 exporting to image
#7 exporting layers done
#7 writing image sha256:e6b4ef3ca4f2cc8516ecac7b9c479fb3c600b3e93d2c0100549f918debca3323 done
#7 naming to docker.io/library/bash-docker done
#7 DONE 0.0s

What's Next?
  View a summary of image vulnerabilities and recommendation

In [25]:
import yaml
import re
import subprocess

In [26]:
def parse_yaml_script_file(file_path):
    with open(file_path) as f:
        try:
            text = f.read()
            parsed_data = yaml.safe_load(text)
            return parsed_data
        except FileNotFoundError:
            print('File not found')
            return None
        except yaml.YAMLError as exc:
            print(exc)
            return None